# Homework 4: Data Acquisition and Ingestion
Name: Joshua Zhong
Date: 08/19/25

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [1]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv


RAW = pathlib.Path('../../project/data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print
# ('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [2]:
print(RAW)

../../project/data/raw


## Helpers (use or modify)

In [5]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Using yahoo finance to pull data about AAPL

In [6]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Close']]
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

/var/folders/dk/pzmx27fj0198nc95vcbfwgn00000gn/T/ipykernel_37932/700782901.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Close']]
[*********************100%***********************]  1 of 1 completed


{'missing': [], 'shape': (64, 2), 'na_total': 0}

In [7]:
# print(dt.datetime.today())
print(df_api.head())

        date   adj_close
0 2025-05-20  206.625504
1 2025-05-21  201.860901
2 2025-05-22  201.131729
3 2025-05-23  195.048645
4 2025-05-27  199.983047


In [8]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved ../../project/data/raw/api_source-yfinance_symbol-AAPL_20250820-104704.csv


## Part 2 — Scrape a Public Table (Required)
Scraping Hockey data from 'scrapethissite.com'

In [9]:
SCRAPE_URL = 'https://www.scrapethissite.com/pages/forms/'  # TODO: replace with permitted page
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

{'missing': [], 'shape': (25, 9), 'na_total': 0}

In [10]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='hockey teams')

Saved ../../project/data/raw/scrape_site-example_table-hockey teams_20250820-104739.csv


## Documentation
- API Source: Yahoo Finance (yfinance) publically available python library. Downloaded past 3 months closing prices for AAPL
- Scrape Source: scrapethissite.com was employed. Scraped a table about NHL team statistics in the year 1990. 
-`.env` is not committed.

## Assumptions and Risks

- Assuming that usage limitations for scraping and API calls are not surpassed
- Websites being down from traffic
- Assuming that table schema doesn't change
- Regulations and legal that prevent our data acquisition